In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [16]:
score_labels = pd.read_csv('./outputs/score_labels.csv')
score_labels

,index,Class,score,video_x
0,1_1_1,1,0.226296,1_1
1,1_1_2,1,0.242073,1_1
2,1_1_3,1,0.258714,1_1
3,1_1_4,1,0.273772,1_1
4,1_1_5,1,0.283500,1_1
...,...,...,...,...
163098,707_3_23,6,1.000000,707_3
163099,707_3_24,6,0.929001,707_3
163100,707_3_25,5,0.829789,707_3
163101,707_3_26,5,0.722700,707_3


In [38]:
labels = []
indexes = []
videos = []
scores = []
for i in range(len(score_labels)):
    if score_labels['video_x'][i].split('_')[1] == '3':
        labels.append(score_labels['Class'][i])
        indexes.append(score_labels['index'][i])
        videos.append(score_labels['video_x'][i])
        scores.append(score_labels['score'][i])

femur_data = pd.DataFrame({'index': indexes, 'Class': labels, 'score' : scores, 'video_x': videos})

In [39]:
femur_data.to_csv('./outputs/femur_data.csv', index = False)

In [40]:
videos = list(femur_data['video_x'].unique())

In [3]:
score_labels = pd.read_csv('./outputs/labels_corrected.csv')

In [9]:
len(score_labels['video'].unique())

1515

In [9]:
im = Image.open('/data/kpusteln/fetal/fetal_extracted/1_1_1.png')

In [2]:
from PIL import Image
import os
import torch
import numpy as np
import torch.distributed as dist
from torchvision import datasets, transforms
#from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
#from timm.data import Mixup
#from timm.data import create_transform

In [8]:
im = im

ValueError: conversion from RGB to GRAYSCALE not supported

In [8]:
import os
import json
import torch.utils.data as data
import numpy as np
from PIL import Image
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import warnings
import torchvision.transforms as transforms
from torchvision.utils import save_image

warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)


class Fetal(data.Dataset):
    def __init__(self, root, ann_path, transform=None, target_transform=None):

        self.data_path = root
        self.ann_path = ann_path
        self.transform = transform
        self.target_transform = target_transform
        # id & label: https://github.com/google-research/big_transfer/issues/7
        # total: 21843; only 21841 class have images: map 21841->9205; 21842->15027
        self.database = pd.read_csv(self.ann_path)

    def _load_image(self, path):
        try:
            im = Image.open(path)
        except:
            print("ERROR IMG LOADED: ", path)
            random_img = np.random.rand(224, 224, 3) * 255
            im = Image.fromarray(np.uint8(random_img))
        return im

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        idb = self.database.iloc[index]

        # images
        images = self._load_image(self.data_path  + idb[0] + '.png')
        if self.transform is not None:
            images = self.transform(images)

        # target
        indexes = idb[0]
        target = idb[1]
        #score = idb[2]
        score = 1
        if self.target_transform is not None:
            target = self.target_transform(target)
        #save_image(images[0], '/data/kpusteln/examples' + str(index) + '.png')
        return images, target, score

    def __len__(self):
        return len(self.database)


In [12]:
t = transforms.Compose([transforms.Resize((450, 600)),
                transforms.Pad((0, 0, 0, 150), fill = 0, padding_mode = 'constant'),
                transforms.Resize((512, 512)),
                transforms.ToTensor(),
                transforms.Normalize(mean=mean, std=std)])

In [13]:
fetal = Fetal(root = '/data/kpusteln/fetal/fetal_extracted/', ann_path = './outputs/labels_corrected.csv', transform=t)

In [16]:
batch_size= 1

In [14]:
from torchvision.utils import save_image

In [17]:
loader = torch.utils.data.DataLoader(
      fetal,
      batch_size=batch_size,
      shuffle=False,
      num_workers=40,
  )

In [18]:
for data in loader:
    data = data[0]
    save_image(data, 'example1.png')
    break

In [309]:
mean

array(0.1447944, dtype=float32)

In [288]:
meansq = var + mean**2

In [289]:
true_meansq = meansq/(224*224)
true_mean = mean/(224*224)

In [290]:
true_std = np.sqrt(true_meansq - true_mean**2)

In [297]:
std

2369.5637

In [6]:
mean_std = np.load('./mean_std.npy')

In [11]:
mean_std
mean = mean_std[0]
std = mean_std[1]

In [20]:
std

0.18222201

In [1]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [2]:
frame_position = [i+1 for i in range(10)]

In [3]:
frame_position

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [3]:
int(355/8) + 355%8

47

In [4]:
import torch

In [5]:
x = torch.randn(64, 3, 224, 224)

In [ ]:
y = torch.randn(1, 8, 2304, 1)

In [ ]:
for i in range(int((64)/8)) :
    x[i*8:(i+1)*8] = self.backbone(x[i*8:(i+1)*8])
x[(i+1)*8:] = self.backbone(x[(i+1)*8:])

In [286]:
import pandas as pd

In [294]:
data = pd.read_csv('./outputs/labels_corrected.csv')

In [290]:
indexes = []
videos = []
for i in range(len(data)):
    row = data.iloc[i]
    idx = row[0]
    label = row[1]
    video = row[2]
    if video == '420_2':
        if label == 5 or label == 6:
            video = video.split('_')
            video[1] = '3'
            video = '_'.join(video)
            idx = idx.split('_')
            idx[1] = '3'
            idx = '_'.join(idx)
            print(idx)
            print(video)
    videos.append(video)
    indexes.append(idx)
            
            
        
        
    

420_3_1
420_3
420_3_2
420_3
420_3_3
420_3
420_3_4
420_3
420_3_5
420_3
420_3_6
420_3
420_3_7
420_3
420_3_8
420_3
420_3_9
420_3
420_3_10
420_3
420_3_11
420_3
420_3_12
420_3
420_3_13
420_3
420_3_14
420_3
420_3_15
420_3
420_3_16
420_3
420_3_17
420_3
420_3_18
420_3
420_3_19
420_3
420_3_20
420_3
420_3_21
420_3
420_3_22
420_3
420_3_23
420_3
420_3_24
420_3
420_3_25
420_3
420_3_26
420_3
420_3_27
420_3
420_3_28
420_3
420_3_29
420_3
420_3_30
420_3
420_3_31
420_3
420_3_32
420_3
420_3_33
420_3
420_3_34
420_3
420_3_35
420_3
420_3_36
420_3
420_3_37
420_3
420_3_38
420_3
420_3_39
420_3
420_3_40
420_3
420_3_41
420_3
420_3_42
420_3
420_3_43
420_3
420_3_44
420_3
420_3_45
420_3
420_3_46
420_3
420_3_47
420_3
420_3_48
420_3
420_3_49
420_3
420_3_50
420_3
420_3_51
420_3
420_3_52
420_3
420_3_53
420_3
420_3_54
420_3
420_3_55
420_3
420_3_56
420_3
420_3_57
420_3
420_3_58
420_3
420_3_59
420_3
420_3_60
420_3
420_3_61
420_3
420_3_62
420_3
420_3_63
420_3
420_3_64
420_3
420_3_65
420_3
420_3_66
420_3
420_3_67
420_3
420_

In [292]:
data['index'] = indexes
data['video'] = videos

In [295]:
len(data['index'].unique())

261527

In [ ]:
data.to_csv('./outputs/labels_corrected.csv', index = False)